In [1]:
# SparkSession - you control your spark app through driver process called SparkSession, which is available as Spark variable
spark

Out[ 1 ]: <pyspark.sql.session.SparkSession at 0x7f5bdea4f3d0>

In [2]:
# One Column Containing 1000 rows with values from 0 to 99
myRange = spark.range(1000).toDF("number")
# Spark DataFrame
# This range of numbers exists on a distributed collection. When run on a cluster,each part of this range of numbers exists on a different executor

## DataFrames
DataFrame is the most common **Structured API**, represents a table of data with rows and columns.
**Schema** is the list that defines the columns and the types of within those columns. Spark has several core abstractions (distriburted collections of data) like Datasets, DataFrames, SQL tables and RDD's.
### Partitions
* to allow every executor to perform work in parallel, spark breaks up the data into partitions (a collection of rows that sit on one physical machine in your cluster)
* if you have one partition and thousand executors or vice versa - spark will have parallelism of only one because you have only one computation resource.
* with DataFrames you don't manipulate manually
## Transformations
In spark, core data structures are immutable, you can modify them - called Transformations
* Narrow transformations - contributes to only on output partition (one to one)
* Wide transdformations - dependencies contributing to many output partitions (one to many)
### Lazy Evaluation
Spark will wait until the very last moment to execute the graph of computation instructions. Instead of modifying the data immediately, saprk build up a plan of transformations til the last minute. This makes spark optimize the entire data flow from end to end.

In [4]:
# transformation 
divisBy2 = myRange.where("number % 2 = 0")
# scala val divisBy2 = myRange.where("number % 2 = 0")
# Note: these return no output bcz we specified only an abstract transformation, spark will not act on transformation until we call an action

<span style="color:brown">some *brown* text</span>.
##Action
transformations allows us to buildup our logical transformation plan. To trigger the computation, we run an action. example: divisBy2**.count()**
* to view in the console
* to collect data to native objects in the respective language
* to write to output data sources
## SparkUI
you can monitor the progress of a job through the Spark web UI, available on port 4040 of the driver node.(if running on local mode http://localhost:4040)
SparkUI - displays the state of your spark jobs, its environment and cluster state. useful for tuning and debugging.
**"Spark job"** represents the transformations triggered by an individual action.

In [6]:
divisBy2.count()

Out[ 4 ]: 500

# An End-to-End Example 
**schema inference**- spark takes the best guess at what the schema of our DataFrame should be.
To get schema information, Spark readsd in a little bit of the data and then attempts to parse the types in those rows according to the types available in spark. Recommended to strictly specify the schema in production scenarios.

In [8]:
flightData2015 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/FileStore/tables/2015_summary-ebaee.csv")

In [9]:
# converted into local array or list of rows - Array(row(...),Row(...))
flightData2015.take(3)

Out[ 6 ]: 
[Row(DEST_COUNTRY_NAME=u'United States', ORIGIN_COUNTRY_NAME=u'Romania', count=15),
 Row(DEST_COUNTRY_NAME=u'United States', ORIGIN_COUNTRY_NAME=u'Croatia', count=1),
 Row(DEST_COUNTRY_NAME=u'United States', ORIGIN_COUNTRY_NAME=u'Ireland', count=344)]

sort doesn't modify the DataFrame, instead that returns a new DataFrame by transforming the previous DataFrame
**CSV ->(read-narrow) DF -> (sort-wide) DF -> (take(3)) Array(...)
Reading -> Sorting -> Collecting a DF.**
sort is wide transformation, bcz rows will need to be compared with one another.

In [11]:
# explain on any DataFrame object, to see how spark will execute this query
flightData2015.sort("count").explain()

== Physical Plan ==
*(2) Sort [count#39 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#39 ASC NULLS FIRST, 200)
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#37,ORIGIN_COUNTRY_NAME#38,count#39] Batched: false, Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>

In [12]:
# By default, when we perform shuffle operation, spark outputs 200 shuffle partitions 
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData2015.sort("count").take(2)

Out[ 8 ]: 
[Row(DEST_COUNTRY_NAME=u'United States', ORIGIN_COUNTRY_NAME=u'Singapore', count=1),
 Row(DEST_COUNTRY_NAME=u'Moldova', ORIGIN_COUNTRY_NAME=u'United States', count=1)]

In [13]:
# By default, when we perform shuffle operation, spark outputs 200 shuffle partitions 
spark.conf.set("spark.sql.shuffle.partitions", "200")
flightData2015.sort("count").take(2)

Out[ 9 ]: 
[Row(DEST_COUNTRY_NAME=u'United States', ORIGIN_COUNTRY_NAME=u'Singapore', count=1),
 Row(DEST_COUNTRY_NAME=u'Moldova', ORIGIN_COUNTRY_NAME=u'United States', count=1)]

**the logical plan that we build up defines a lineage for the DF, so that at any given point of time Spark know how to recompute any partition by performing all of the operations it had before on the same data (Functional programming is at the spark's core)**

###DataFrames and SQL
You can express your business logic in SQL or DataFrames (either in R,Scala,Python, or Java) and Spark will compile that logic down to an Underlying plan(that you can see in the explain plan)before actually executing the code.
**No performance difference between writing SQL queries or writing DataFrame code, they both "compile" to the same underlying plan that we specify in the DF code**

In [16]:
#can make any df into a table or view with one simple method call
flightData2015.createOrReplaceTempView("flight_data_2015")

In [17]:
# the SQL query against a df returns another df - Powerful, as it is possible for you to  specify the transformations in the manner you want 
# without compromising on efficency. Below the two physical plans are exactly the same
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()

sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#37], functions=[finalmerge_count(merge count#70L) AS count(1)#58L])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#37, 200)
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#37], functions=[partial_count(1) AS count#70L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#37] Batched: false, Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#37], functions=[finalmerge_count(merge count#72L) AS count(1)#65L])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#37, 200)
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#37], functions=[partial_count(1) AS count#72L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#37] Batched: false, Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

In [18]:
from pyspark.sql.functions import max

flightData2015.select(max("count")).take(1)


Out[ 12 ]: [Row(max(count)=370002)]

In [19]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

maxSql.show()

+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
+-----------------+-----------------+

In [20]:
from pyspark.sql.functions import desc

flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()


+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
+-----------------+-----------------+

In [21]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()


== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#129L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#37,destination_total#129L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#37], functions=[finalmerge_sum(merge sum#135L) AS sum(cast(count#39 as bigint))#125L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#37, 200)
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#37], functions=[partial_sum(cast(count#39 as bigint)) AS sum#135L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#37,count#39] Batched: false, Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>

**Spark lazily executes a DAG of transformations in order to optimize the execution plan on DF's. **